In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

env = "production"
#s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-03-14/hr=16/part-00000-76298f84-5414-49c5-88f6-a981ff06d94d-c000.csv .

s3_bucket = "s3://mist-aggregated-stats-{env}/aggregated-stats/".format(env=env)

date_day = "2020-04-04"
hr = '20'

rrm_local_path = "top_1_time_epoch_by_site_ap_ap2_band/dt={day}/hr={hr}/*.csv".format(env=env, day=date_day, hr=hr)

rrm_local_path = s3_bucket + rrm_local_path
print(rrm_local_path)



# user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
# user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# # user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
# user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-aggregated-stats-production/aggregated-stats/top_1_time_epoch_by_site_ap_ap2_band/dt=2020-04-04/hr=20/*.csv


In [2]:
rdd_rrm_local = spark.sparkContext.textFile(rrm_local_path)
rdd_rrm_local.take(3)

['site,ap,ap2,band,time_epoch,rssi',
 '0009dab3-7a6b-4ace-85ff-e6dc20768bcb,5c5b358e2846,5c5b358e26e3,5,1586033886,-43.0',
 '00252227-5f8f-4e6b-a74b-182eaa173271,5c5b358e4669,5c5b352e8de0,5,1586033884,-66.0']

In [3]:
# rdd_rrm_local.map(lambda line: (line.split(',')[0], line.split(',')[1])).collect()

In [4]:

df_scan = spark.read.format("csv")\
   .option("header", "true")\
   .option("inferSchema", "true")\
   .load(rrm_local_path)
df_scan.createOrReplaceTempView("scan_data")
df_Schema = df_scan.schema

In [5]:
df_Schema

StructType(List(StructField(site,StringType,true),StructField(ap,StringType,true),StructField(ap2,StringType,true),StructField(band,IntegerType,true),StructField(time_epoch,IntegerType,true),StructField(rssi,DoubleType,true)))

In [6]:
df_scan.show()

+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|0009dab3-7a6b-4ac...|5c5b358e2846|5c5b358e26e3|   5|1586033886|-43.0|
|00252227-5f8f-4e6...|5c5b358e4669|5c5b352e8de0|   5|1586033884|-66.0|
|00272816-8dc2-4f1...|5c5b357f3308|5c5b35ae6d5b|   5|1586033864|-57.0|
|00272816-8dc2-4f1...|5c5b357f33d0|5c5b35ae6db5|   5|1586033864|-69.0|
|00272816-8dc2-4f1...|5c5b35ae6d83|5c5b35ae6dc9|  24|1586033865|-59.0|
|00272816-8dc2-4f1...|5c5b35ae6e05|        null|   5|1586033862|-68.0|
|00458c29-d67b-4cf...|5c5b352ee1b4|5c5b353e6469|  24|1586032915|-81.0|
|00458c29-d67b-4cf...|5c5b353e5708|5c5b353e569a|   5|1586033895|-68.0|
|00458c29-d67b-4cf...|5c5b353e574e|5c5b353e5708|  24|1586033906|-75.0|
|00458c29-d67b-4cf...|5c5b353e574e|5c5b35ceffdc|  24|1586033906|-81.0|
|00458c29-d67b-4cf...|5c5b353e6464|5c5b353e565e|  24|1586033902|-49.0|
|00458

In [7]:
df_scan.describe().show()

+-------+--------------------+------------+------------+-----------------+--------------------+------------------+
|summary|                site|          ap|         ap2|             band|          time_epoch|              rssi|
+-------+--------------------+------------+------------+-----------------+--------------------+------------------+
|  count|             2688177|     2688177|     2465367|          2688177|             2688177|           2688177|
|   mean|                null|        null|        null| 16.1225800979623|1.5860336806575158E9|-66.59282777882558|
| stddev|                null|        null|        null|9.360409521984565|   613.7852824062135| 8.444705546303947|
|    min|0009dab3-7a6b-4ac...|5c5b350e011c|5c5b350e0090|                5|          1586030400|            -101.0|
|    max|fffb6917-ee0e-45c...|d420b080bdab|d420b080f145|               24|          1586033998|             -11.0|
+-------+--------------------+------------+------------+-----------------+------

In [8]:
df_scan.count()

2688177

In [9]:
site_id = "d1ee1d22-4b55-4c97-97c4-9d757144f45b"

In [10]:
df_scan_site = df_scan.filter(col('site')==site_id)

In [11]:
df_scan_site.count()

189

In [12]:
df_scan_site.show()

+--------------------+------------+------------+----+----------+-----+
|                site|          ap|         ap2|band|time_epoch| rssi|
+--------------------+------------+------------+----+----------+-----+
|d1ee1d22-4b55-4c9...|5c5b3552b660|        null|   5|1586033944|-76.0|
|d1ee1d22-4b55-4c9...|5c5b3552b660|5c5b3552ba70|   5|1586033760|-63.0|
|d1ee1d22-4b55-4c9...|5c5b3552b714|5c5b3552b94e|   5|1586033812|-63.0|
|d1ee1d22-4b55-4c9...|5c5b3552b7cd|        null|   5|1586033969|-76.0|
|d1ee1d22-4b55-4c9...|5c5b3552b7cd|5c5b3552b70a|   5|1586033787|-65.0|
|d1ee1d22-4b55-4c9...|5c5b3552b5e3|5c5b3552b796|   5|1586033829|-65.0|
|d1ee1d22-4b55-4c9...|5c5b3552b796|5c5b3552b660|   5|1586033816|-64.0|
|d1ee1d22-4b55-4c9...|5c5b3552b94e|5c5b3552b96c|  24|1586033794|-67.0|
|d1ee1d22-4b55-4c9...|5c5b3552ba70|5c5b3552b94e|   5|1586033826|-64.0|
|d1ee1d22-4b55-4c9...|5c5b3552b96c|5c5b3552b94e|  24|1586033782|-63.0|
|d1ee1d22-4b55-4c9...|5c5b3552ba70|5c5b3552b70a|   5|1586033826|-64.0|
|d1ee1

# GraphFrames
The following command creates your first GraphFrame. The GraphFrame accepts two DataFrames as inputs—vertices and edges. GraphFrames like to have a naming convention in the column name, which you need to follow. Those rules are defined as follows:
* A DataFrame that represents vertices should contain a column named id. Here, personsDf contains a column name "id".

* A DataFrame that represents edges should contain columns named src and dst. Here, reationshipDf contains the columns "src" and "dst".

In [13]:
vertices= df_scan_site.selectExpr("ap as id").distinct()

In [14]:
edges=df_scan_site.selectExpr("ap as src", "ap2 as dst", "rssi as weight")

In [15]:
from graphframes import *
g = GraphFrame(vertices, edges)

In [16]:
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

+------------+
|          id|
+------------+
|5c5b3552b796|
|5c5b3552ba70|
|d420b000a866|
|5c5b3552b96c|
|5c5b3552bb6f|
|5c5b3552b70a|
|5c5b3552b714|
|5c5b3552b94e|
|5c5b3552b5e3|
|5c5b3552b7cd|
|5c5b3552b660|
+------------+

+------------+------------+------+
|         src|         dst|weight|
+------------+------------+------+
|5c5b3552b660|        null| -76.0|
|5c5b3552b660|5c5b3552ba70| -63.0|
|5c5b3552b714|5c5b3552b94e| -63.0|
|5c5b3552b7cd|        null| -76.0|
|5c5b3552b7cd|5c5b3552b70a| -65.0|
|5c5b3552b5e3|5c5b3552b796| -65.0|
|5c5b3552b796|5c5b3552b660| -64.0|
|5c5b3552b94e|5c5b3552b96c| -67.0|
|5c5b3552ba70|5c5b3552b94e| -64.0|
|5c5b3552b96c|5c5b3552b94e| -63.0|
|5c5b3552ba70|5c5b3552b70a| -64.0|
|5c5b3552b94e|5c5b3552b70a| -61.0|
|5c5b3552b5e3|        null| -52.0|
|5c5b3552b660|5c5b3552b796| -63.0|
|5c5b3552b5e3|5c5b3552ba70| -53.0|
|5c5b3552ba70|5c5b3552b7cd| -62.0|
|d420b000a866|5c5b3552b714| -83.0|
|5c5b3552b660|5c5b3552b70a| -61.0|
|5c5b3552b660|5c5b3552b714| -63.0|
|5c5

In [17]:
display(g)

GraphFrame(v:[id: string], e:[src: string, dst: string ... 1 more field])

In [18]:
# The following code gets the number of incoming links to Andrew. This is obtained by calling the inDegrees method.

g.degrees.filter("id = '5c5b3552b5e3'").show()

+------------+------+
|          id|degree|
+------------+------+
|5c5b3552b5e3|    37|
+------------+------+



In [22]:
# The following code shows how to get the number of links coming out from Andrew using the outDegrees method .

g.outDegrees.filter("id = '5c5b3552b5e3'").show()


+------------+---------+
|          id|outDegree|
+------------+---------+
|5c5b3552b5e3|       19|
+------------+---------+



In [21]:
sc.setCheckpointDir('graphframes_cps')


In [20]:
ranks = g.pageRank(resetProbability=0.10, maxIter=5)
# display(ranks.vertices.select("id","pagerank").orderBy(desc("pagerank")))


In [22]:
display(ranks.vertices)
display(ranks.edges)

DataFrame[id: string, pagerank: double]

DataFrame[src: string, dst: string, weight: double, weight: double]

In [23]:
# Search from "Esther" for users of age < 32.

ap1 = "5c5b3552b96c"
ap2 = "5c5b3552b5e3"
paths = g.bfs("id = ap1", "id = ap2")
paths.show()

# Specify edge filters or max path lengths.
g.bfs("id = ap1", "id = ap2",\
  edgeFilter="rssi > -75", maxPathLength=3)


AnalysisException: "cannot resolve '`ap1`' given input columns: [id]; line 1 pos 5;\n'Filter (id#463 = 'ap1)\n+- Deduplicate [id#463]\n   +- Project [ap#11 AS id#463]\n      +- Filter (site#10 = d1ee1d22-4b55-4c97-97c4-9d757144f45b)\n         +- Relation[site#10,ap#11,ap2#12,band#13,time_epoch#14,rssi#15] csv\n"

In [ ]:
# g.connectedComponents()

result1 = g.stronglyConnectedComponents(maxIter=10)
result1.select("id", "component").orderBy("component").show()

In [ ]:
# result1.

In [ ]:
result2 = g.stronglyConnectedComponents(maxIter=10)
result2.select("id", "component").orderBy("component").show()

In [ ]:
result2

In [ ]:
date_day + hr

In [ ]:
s3_out_bucket = "s3://mist-test-bucket/" #top_1_time_epoch_by_site_ap_ap2_band/{}/{}/".format(date_day, hr)
s3_out_bucket_vertices = s3_out_bucket #+ "vertices/"
s3_out_bucket_edges = s3_out_bucket #+ "edges/"

print(s3_out_bucket_vertices, s3_out_bucket_edges)
g.vertices.write.parquet(s3_out_bucket_vertices)
g.edges.write.parquet(s3_out_bucket_vertices)
# s3_out_bucket


In [ ]:
# s3_out_bucket, rrm_local_path

In [ ]:
import networkx as nx
gp = nx.from_pandas_edgelist(edges.toPandas(),'src','dst')
nx.draw(gp, with_labels = True)

In [ ]:
g.persist()

In [ ]:
# graph.degrees.filter("id = 1").show()

In [ ]:
# GraphFrame provides the following built-in algorithms:
# Connected components

# Label propagation

# PageRank

# SVD++

# Shortest Path

# Strongly connected components

# Triangle count